In [2]:

import os
import torch
from torch.utils.data import Dataset, DataLoader
import pydicom
import numpy as np
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn as nn
from torch.utils.data import random_split
from model import MedicalImageCNN, CNNToRNA,CNNClassifier, train_model
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

from helper import collate_fn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
import torch
import torch.nn.functional as F

def evaluate_model_auc(model, dataloader, device):
    model.eval()
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for images, _, labels, _ in dataloader:
            images = images.to(device)
            labels = labels.to(device).long()  # ensure labels are class indices

            outputs = model(images)  # shape: [B, num_classes]
            probs = F.softmax(outputs, dim=1)  # shape: [B, num_classes]

            all_probs.append(probs.cpu())
            all_targets.append(labels.cpu())

    probs = torch.cat(all_probs).numpy()   # shape: [N, num_classes]
    targets = torch.cat(all_targets).numpy().squeeze()  # shape: [N]

    if len(probs.shape) != 2:
        raise ValueError(f"Predicted probabilities must be a 2D array. Got shape: {probs.shape}")
    if len(targets.shape) != 1:
        raise ValueError(f"Targets must be a 1D array. Got shape: {targets.shape}")

    try:
        # Use multi_class only if more than 2 classes
        if probs.shape[1] == 2:
            auc = roc_auc_score(targets, probs[:, 1])  # class 1 probability
        else:
            auc = roc_auc_score(targets, probs, multi_class='ovo', average='macro')
    except ValueError as e:
        print(f"Error calculating AUC: {e}")
        auc = None 
    return auc

: 

In [ ]:
print("Top 10 genes:")

#load the gene performance data from the saved file
if not os.path.exists('gene_performance.pth'):
    raise FileNotFoundError("The file 'gene_performance.pth' does not exist.")
gene_performance = torch.load('gene_performance.pth', weights_only=False)
gene_performance = dict(sorted(gene_performance.items(), key=lambda item: item[1]['total_lost']))


#sort by total_lost in descending order


for gene in gene_performance.keys():
    #recalculate the test accuracy
    model = gene_performance[gene]['model']
    model.to(device)
    test_loader = DataLoader(gene_performance[gene]['test_dataset'], batch_size=1, shuffle=False, collate_fn=collate_fn)
    gene_performance[gene]['test_accuracy'] = evaluate_model_auc(gene_performance[gene]['model'], test_loader, device)
    print(gene, gene_performance[gene]['total_lost'], gene_performance[gene]['test_accuracy'])


Top 10 genes:
Error calculating AUC: y should be a 1d array, got an array of shape (20, 2) instead.
ORAI2 0.2984073036595395 nan
Error calculating AUC: y should be a 1d array, got an array of shape (20, 2) instead.
DNAL1 0.3143503289473684 nan
Error calculating AUC: y should be a 1d array, got an array of shape (20, 2) instead.
CHD7 0.4294240851151316 nan
